1. Restart Kernel, Run All 
2. Modify C2 (3 rerun cells)
3. Modify C4 (1 rerun cell)

# Black Scholes from split annotations 
- Computes the Black Scholes formula over a set of vectors, with the `composer` runner from split_annotations off. 
- This is one workload where split annotations worked very well on. I think this is because they are able to split the computation workload by the array passed in. This is a manipulation on splitting the data (vs. finding the right order)
- Source: https://github.com/weld-project/split-annotations/blob/master/python/benchmarks/blackscholes/blackscholes.py
- Data: generated in code

In [1]:
import sys
import argparse
import math
import scipy.special as ss
import time
import numpy as np

## Notes: 
- This is a version of the code more like notebooks.

In [6]:
# SET PARAMETERS HERE: 
size = 40 # Modify(2) Change to 100
threads = 1
piece_size = 16384

In [7]:
# C3
# Generate data and set up variables related to equation.
print("generating data...")
price = np.ones(size, dtype="float64") * 4.0
strike = np.ones(size, dtype="float64") * 3.0
t = np.ones(size, dtype="float64") * 2.2
rate = np.ones(size, dtype="float64") * 8.3
vol = np.ones(size, dtype="float64") * 1.2

c05 = 3.0
c10 = 1.5
invsqrt2 = 1.0 / math.sqrt(2.0)

generating data...


In [10]:
# Allocating memory 
start = time.time() # Modify(3) Change to 0
tmp = np.ones(len(price), dtype="float64")
vol_sqrt = np.ones(len(price), dtype="float64")
rsig = np.ones(len(price), dtype="float64")
d1 = np.ones(len(price), dtype="float64") 
d2 = np.ones(len(price), dtype="float64")
call = np.ones(len(price), dtype="float64")
put = np.ones(len(price), dtype="float64")
end = time.time()
print("Allocation:", end - start)

Allocation: 1745418112.986169


In [11]:
# Outputs

start = time.time()

np.multiply(vol, vol, out=rsig)
np.multiply(rsig, c05, out=rsig)
np.add(rsig, rate, out=rsig)

np.sqrt(t, out=vol_sqrt)
np.multiply(vol_sqrt, vol, out=vol_sqrt)

np.multiply(rsig, t, out=tmp)
np.divide(price, strike, out=d1)
np.log2(d1, out=d1)
np.add(d1, tmp, out=d1)

np.divide(d1, vol_sqrt, out=d1)
np.subtract(d1, vol_sqrt, out=d2)

# d1 = c05 + c05 * erf(d1 * invsqrt2)
np.multiply(d1, invsqrt2, out=d1)

ss.erf(d1, out=d1)

np.multiply(d1, c05, out=d1)
np.add(d1, c05, out=d1)

# d2 = c05 + c05 * erf(d2 * invsqrt2)
np.multiply(d2, invsqrt2, out=d2)


ss.erf(d2, out=d2)

np.multiply(d2, c05, out=d2)
np.add(d2, c05, out=d2)

# Reuse existing buffers
e_rt = vol_sqrt
tmp2 = rsig

# e_rt = exp(-rate * t)
np.multiply(rate, -1.0, out=e_rt)
np.multiply(e_rt, t, out=e_rt)
np.exp(e_rt, out=e_rt)

# call = price * d1 - e_rt * strike * d2
#
# tmp = price * d1
# tmp2 = e_rt * strike * d2
# call = tmp - tmp2
np.multiply(price, d1, out=tmp)
np.multiply(e_rt, strike, out=tmp2)
np.multiply(tmp2, d2, out=tmp2)
np.subtract(tmp, tmp2, out=call)

# put = e_rt * strike * (c10 - d2) - price * (c10 - d1)
# tmp = e_rt * strike
# tmp2 = (c10 - d2)
# put = tmp - tmp2
# tmp = c10 - d1
# tmp = price * tmp
# put = put - tmp
np.multiply(e_rt, strike, out=tmp)
np.subtract(c10, d2, out=tmp2)
np.multiply(tmp, tmp2, out=put)
np.subtract(c10, d1, out=tmp)
np.multiply(price, tmp, out=tmp)
np.subtract(put, tmp, out=put)

end = time.time()
print("Build time:", end - start)

end = time.time()
print("Runtime:", end - start)

print("call: ", call) 
print("put: ", put)

Build time: 0.0015139579772949219
Runtime: 0.0018727779388427734
call:  [23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979 23.99999979
 23.99999979 23.99999979 23.99999979 23.99999979 23.